<a href="https://colab.research.google.com/github/panashematsaudza/Pyspark-Hyundai-Cruise-Ships-/blob/master/Hyundai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyspark.sql import SparkSession


In [0]:
spark = SparkSession.builder.appName('Hyndai Cruise').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
data = spark.sql("SELECT * FROM cruise_ship")

In [0]:
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [0]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [0]:
data.head(1)

Out[7]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [0]:
data.columns

Out[9]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler

In [0]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="shipNameIndex")
indexed = indexer.fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|shipNameIndex|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------+
only showing top 20 rows

In [0]:
indexed.columns

Out[17]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'shipNameIndex']

In [0]:
assembler = VectorAssembler(inputCols = [ 'Age','Tonnage', 'passengers','length', 'cabins','passenger_density','shipNameIndex'],outputCol ='features')

In [0]:
output = assembler.transform(indexed)


In [0]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------+--------------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|shipNameIndex| features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------+--------------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[6.0,30.276999999...|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[6.0,30.276999999...|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|[26.0,47.262,14.8...|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|[11.0,110.0,29.74...|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|[17.0,101.353,26....|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[22.0,70.367,20.5...|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[15.0,70.367,20.5...|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|[23.0,70.367,20.5...|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[19.0,70.367,20.5...|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|[6.0,110.23899999...|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[10.0,110.0,29.74...|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|[28.0,46.052,14.5...|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[18.0,70.367,20.5...|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[17.0,70.367,20.5...|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|[11.0,86.0,21.24,...|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[8.0,110.0,29.74,...|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|[9.0,88.5,21.24,9...|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[15.0,70.367,20.5...|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|[12.0,88.5,21.24,...|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[20.0,70.367,20.5...|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------+--------------------+
only showing top 20 rows

In [0]:
output.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- shipNameIndex: double (nullable = false)
-- features: vector (nullable = true)

In [0]:
output.select('features').show()


+--------------------+
 features|
+--------------------+
[6.0,30.276999999...|
[6.0,30.276999999...|
[26.0,47.262,14.8...|
[11.0,110.0,29.74...|
[17.0,101.353,26....|
[22.0,70.367,20.5...|
[15.0,70.367,20.5...|
[23.0,70.367,20.5...|
[19.0,70.367,20.5...|
[6.0,110.23899999...|
[10.0,110.0,29.74...|
[28.0,46.052,14.5...|
[18.0,70.367,20.5...|
[17.0,70.367,20.5...|
[11.0,86.0,21.24,...|
[8.0,110.0,29.74,...|
[9.0,88.5,21.24,9...|
[15.0,70.367,20.5...|
[12.0,88.5,21.24,...|
[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows

In [0]:
final_data = output.select('features','crew')

In [0]:
final_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
train_data ,test_data = final_data.randomSplit([0.7,0.3])


In [0]:
train_data.describe().show()


+-------+------------------+
summary| crew|
+-------+------------------+
 count| 111|
 mean| 7.53144144144145|
 stddev|3.4291172590165493|
 min| 0.59|
 max| 21.0|
+-------+------------------+

In [0]:
lr = LinearRegression(labelCol = 'crew')


In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)


In [0]:
test_results.residuals.show()


+--------------------+
 residuals|
+--------------------+
 0.8469981542841003|
 -1.3536495624329756|
 -0.6204093974401799|
 -0.3258098509857472|
 0.33190890194222256|
-0.19399904565822013|
 -0.633820838000851|
 -0.5289368343192447|
 0.5111278686050174|
 -0.5735393072214698|
 -0.5117287036183402|
-0.38571497786775666|
-0.14962115846823743|
 -0.5053999042944479|
 1.6860212812332431|
 0.24240275462570438|
 1.0351590269177553|
-0.37618423636376086|
 6.9901034832619935|
 -1.039361944498559|
+--------------------+
only showing top 20 rows

In [0]:
test_results.rootMeanSquaredError

Out[34]: 1.2989223969228472

In [0]:
test_results.r2

Out[35]: 0.8695433275265768

In [0]:
final_data.describe().show()


+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 158|
 mean|7.794177215189873|
 stddev|3.503486564627034|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [0]:
unlabeled_data = test_data.select('features')


In [0]:
preds = lr_model.transform(unlabeled_data)


In [0]:
preds.show()


+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[5.0,122.0,28.5,1...| 5.8530018457159|
[5.0,160.0,36.34,...|14.953649562432975|
[6.0,30.276999999...| 4.17040939744018|
[6.0,112.0,38.0,9...|11.225809850985748|
[6.0,113.0,37.82,...|11.668091098057777|
[6.0,158.0,43.7,1...| 13.79399904565822|
[7.0,116.0,31.0,9...|12.633820838000851|
[8.0,110.0,29.74,...|12.128936834319244|
[9.0,81.0,21.44,9...| 9.488872131394983|
[9.0,90.09,25.01,...| 9.26353930722147|
[9.0,105.0,27.2,8...| 11.19172870361834|
[10.0,86.0,21.14,...| 9.585714977867756|
[10.0,91.62700000...| 9.149621158468237|
[10.0,110.0,29.74...|12.105399904294448|
[10.0,151.4,26.2,...|10.843978718766756|
[11.0,58.6,15.66,...| 7.357597245374295|
[11.0,90.0,22.4,9...| 9.964840973082245|
[11.0,90.09,25.01...| 8.856184236363761|
[11.0,110.0,29.74...|12.109896516738008|
[11.0,138.0,31.14...|12.889361944498559|
+--------------------+------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import corr

In [0]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [0]:
data.select(corr('crew','cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+